In [2]:
import pandas as pd # 판다스 라이브러리, 데이터 조작
import numpy as np # 넘파이 라이브러리, 데이터 연산
import matplotlib.pylab as plt # 그래프 출력 라이브러리

In [3]:
def sum_squares_error(y, t): # y 확률로 된 예측 데이터 , t 원-핫 인코딩으로 되어진 정답 레이블
    return 0.5 * np.sum(( y - t ) ** 2 )

# 예측의 정확도를 반환함 (확률로 반환)

In [4]:
def cross_entropy_error(y, t): # y 확률로 된 예측 데이터 , t 원-핫 인코딩으로 되어진 정답 레이블
    delta = 1e-7
    return -np.sum( t * np.log( y + delta ))

# 정답에 가깝게 예측할수록 오차가 적다 (적은 오차를 수치로 반환)
# y = log_x 그래프는 [ ~ 0] 범위를 가지고 있어서 -를 붙여서 반전한다. ( 오차 값이기 때문에 +로 표현 )

In [5]:
def cross_entropy_error(y, t):
    if y.ndim == 1: # ndim : 차원
        t = t.reshape(1, t.size) # reshape(1, t.size) : 다차원 배열을 한 차원으로 변경
        y = y.reshape(1, y.size) # y.size : y 배열의 개수 반환
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1) # 세로 : 행(0), 가로 : 열(1) 기준으로 최대값을 저장
             
    batch_size = y.shape[0] # x 개수 반환
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size # 로그 반환

In [6]:
def numerical_diff(f, x):
    h = 1e-4
    return ( f(x+h) - f(x-h) ) / ( 2 * h )

In [7]:
def numerical_gradient(f, x): # 편미분을 벡터로 정리 : 기울기
    h = 1e-4
    grad = np.zeros_like(x)

    for idx in range(x.size):
        tmp_val = x[idx]

        x[idx] = tmp_val + h # (구하고싶은 값+1) 미분
        fxh1 = f(x)

        x[idx] = tmp_val - h # # (구하고싶은 값-1) 미분
        fxh2 = f(x)

        # 두 미분 값의 평균으로 구하고 싶은 값의 미분 값을 구한다.
        grad[idx] = ( fxh1 - fxh2 ) / ( 2 * h )
        x[idx] = tmp_val
    
        return grad

In [8]:
# 경사 하강법
def gradient_descent(f, init_x, lr=0.01, step_num=100): # 최적화 하려는 함수, 초깃값, 학습률, 반복횟수
    x = init_x

    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    
    return x

In [9]:
def softmax(x): # 모든 신호에 영향을 준다.
    if x.ndim == 2: # 차원이 2차원일때
        x = x.T # array.T : 배열의 행과 열을 바꾸는 메소드
        x = x - np.max(x, axis=0) # 세로(0) 행 방향의 최대값을 저장
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

In [10]:
class simpleNet: # 기울기 구하는 클래스
    
    def __init__(self): # 표준 정규 분포
        self.W = np.random.randn(2,3)
    
    def predict(self, x):
        return np.dot(x, self.W) # np.dot : 행렬의 곱의 합
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

In [11]:
def f(W):
    return net.loss(x, t)

In [12]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x)) # 자연스러운 곡선을 그리는 함수

In [13]:
# 2층 신경망 클래스 구현
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): # weight_init_std : 학습률
        self.params = {} # 배치처리를 위해 2층의 신경망 구현
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) # 학습률에 따른 변수 조정
        self.params['b1'] = np.zeros(hidden_size) # shape는 같지만 모든 요소ㄱ 0으로 되어진 배열 생성
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) # 학습률에 따른 변수 조정
        self.params['b2'] = np.zeros(output_size) # shape는 같지만 모든 요소ㄱ 0으로 되어진 배열 생성

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1) # 1층 활성화함수 : Sigmoid(시그모이드)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2) # 2층 활성화함수 : SoftMax(소프트맥스)

        return y
    
    def loss(self, x, t): # 오차
        y = self.predict(x)
    
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t): # 정확도
        y = self.predict(x)
        y = np.argmax(y, axis=1) # 최대값 반환
        t = np.argmax(t, axis=1) # 최대값 반환

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t): # 경사 하강법
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

In [15]:
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

iters_num = 10000 # 반복횟수
train_size = x_train.shape[0] # 학습 사이즈
batch_size = 100 # 미니 배치 크기
learning_rate = 0.1 # 학습률

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.numerical_gradient(x_batch, t_batch)
    # 개선판은 다음 단원에서 공부

    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

In [17]:
 from dataset.mnist import load_mnist # pkl 파일 사용

 (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

 network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

 iters_num = 10000
 train_size = x_train.shape[0]
 batch_size = 100
 learning_rate = 0.1

 train_loss_list = []
 train_acc_list = []
 test_acc_list = []
                           # 60000 / 100
 iter_per_epoch = max(train_size / batch_size, 1)

 for i in range(iters_num):
     batch_mask = np.random.choice(train_size, batch_size)
     x_batch = x_train[batch_mask]
     t_batch = t_train[batch_mask]

     grad = network.numerical_gradient(x_batch, t_batch)

     for key in ('W1', 'b1', 'W2', 'b2'):
         network.params[key] -= learning_rate * grad[key]

     loss = network.loss(x_batch, t_batch)
     train_loss_list.append(loss)

     if i % iter_per_epoch == 0:
         train_acc = network.accuracy(x_train, t_train)
         test_acc = network.accuracy(x_test, t_test)
         train_acc_list.append(train_acc)
         test_acc_list.append(test_acc)
         print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
train acc, test acc | 0.10441666666666667, 0.1028
